<a href="https://colab.research.google.com/github/rijul57/Rijul_BTP-Named_Entity_Recoginition/blob/main/Naamapadam_Train_BMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets seqeval scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b7bc9a191b5f235a9e39ed546deba1a5719d6f9449f44413c162ff49c41ca3a6
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency reso

In [ ]:
from datasets import Dataset, DatasetDict

def load_conll_data(filepath):
    tokens, ner_tags = [], []
    with open(filepath, encoding='utf-8') as f:
        words, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    tokens.append(words)
                    ner_tags.append(tags)
                    words, tags = [], []
                continue
            splits = line.split()
            if len(splits) != 2: continue
            word, tag = splits
            words.append(word)
            tags.append(tag)
        if words:
            tokens.append(words)
            ner_tags.append(tags)
    return Dataset.from_dict({"tokens": tokens, "ner_tags": ner_tags})


In [ ]:
train_ds = load_conll_data("Bhojpuri-IndicNER_train.txt")
test_ds = load_conll_data("Bhojpuri-IndicNER.txt")
dataset = DatasetDict({"train": train_ds, "test": test_ds})


In [ ]:
unique_tags = set(tag for doc in dataset["train"]["ner_tags"] for tag in doc)
label2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2label = {i: tag for tag, i in label2id.items()}

dataset = dataset.map(lambda x: {"labels": [[label2id[tag] for tag in seq] for seq in x["ner_tags"]]}, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "ai4bharat/IndicNER"  # This is the official model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/ai4bharat/IndicNER.
401 Client Error. (Request ID: Root=1-691ca487-770e8d2915ef091464648262;905e1e96-56f9-412d-9e33-ee40e85a98ef)

Cannot access gated repo for url https://huggingface.co/ai4bharat/IndicNER/resolve/main/config.json.
Access to model ai4bharat/IndicNER is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(example["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                aligned.append(-100)
            elif word_id != previous_word_id:
                aligned.append(label[word_id])
            else:
                aligned.append(label[word_id] if label[word_id] != -100 else -100)
            previous_word_id = word_id
        labels.append(aligned)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


In [ ]:
import evaluate
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_preds = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_preds, references=true_labels)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification

args = TrainingArguments(
    output_dir="./bhojpuri-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
trainer.save_model("./bhojpuri-ner")
tokenizer.save_pretrained("./bhojpuri-ner")

In [ ]:
output = trainer.predict(tokenized_datasets["test"])
predictions = output.predictions
labels = output.label_ids
pred_labels = predictions.argmax(axis=-1)

flat_preds = [id2label[i] for row, lab in zip(pred_labels, labels) for i, l in zip(row, lab) if l != -100]
from collections import Counter
print(Counter(flat_preds))


In [ ]:
print("Sample tokens:", dataset["train"][0]["tokens"][:50])
print("NER tags:", dataset["train"][0]["ner_tags"][:50])
print("Label IDs:", dataset["train"][0]["labels"][:50])

In [ ]:
from collections import Counter

all_labels = [label for example in dataset["train"]["labels"] for label in example]
print(Counter(all_labels))


Counter({6: 154313, 2: 3689, 5: 1449, 0: 620, 3: 70, 4: 45, 1: 40})


In [ ]:
def has_entity(example):
    return any(label != 6 for label in example["labels"])

filtered_dataset = dataset.filter(has_entity)

Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Print the first example's labels to check the structure
print(dataset['train'][0]['labels'][:50])


[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [ ]:
print(len(filtered_dataset["train"]))

1


In [ ]:
print("Original train size:", len(dataset["train"]))
print("Filtered train size:", len(filtered_dataset["train"]))

Original train size: 1
Filtered train size: 1


In [ ]:
from datasets import Dataset, DatasetDict

# Define sentence-ending punctuation marks
SENTENCE_END_TOKENS = {"।", "?", "!"}

def split_into_sentences(example):
    sentences = []
    sentence = {"tokens": [], "labels": []}

    for token, label in zip(example["tokens"], example["labels"]):
        sentence["tokens"].append(token)
        sentence["labels"].append(label)

        if token in SENTENCE_END_TOKENS:
            sentences.append(sentence)
            sentence = {"tokens": [], "labels": []}

    # If any leftover sentence remains (no ending punctuation)
    if sentence["tokens"]:
        sentences.append(sentence)

    return {
        "tokens": [s["tokens"] for s in sentences],
        "labels": [s["labels"] for s in sentences]
    }

def flatten_sentences(dataset_split):
    split_data = dataset_split.map(split_into_sentences, batched=False)

    # Flatten nested sentence lists
    all_tokens = sum(split_data["tokens"], [])
    all_labels = sum(split_data["labels"], [])

    return Dataset.from_dict({"tokens": all_tokens, "labels": all_labels})

# Apply to each split
new_dataset = DatasetDict({
    split: flatten_sentences(dataset[split])
    for split in dataset
})

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
VALID_ENDINGS = {"।", "?", "!"}

def is_proper_sentence(example):
    tokens = example["tokens"]
    # A proper sentence must end with a valid end token
    return tokens[-1] in VALID_ENDINGS

# Check how many examples end properly
for split in new_dataset:
    data = new_dataset[split]
    total = len(data)
    valid_endings = sum(is_proper_sentence(ex) for ex in data)

    print(f"Split: {split}")
    print(f" - Total examples: {total}")
    print(f" - Sentences ending properly: {valid_endings}")
    print(f" - Improper endings: {total - valid_endings}")

    # Print a few problematic ones (if any)
    for i in range(min(5, total)):
        ex = data[i]
        if not is_proper_sentence(ex):
            print(f"\n[Improper sentence {i}]")
            print("Tokens:", ex["tokens"])
            print("Labels:", ex["labels"])


Split: train
 - Total examples: 10370
 - Sentences ending properly: 10370
 - Improper endings: 0
Split: test
 - Total examples: 4486
 - Sentences ending properly: 4486
 - Improper endings: 0


In [ ]:
print(new_dataset["train"][0])

{'tokens': ['दोसर', 'ई', 'कि', 'ई', 'पूरा', 'कहानी', 'कल्पना', 'ह', ',', 'कवनो', 'कारण', 'से', 'ई', 'सही', 'साबित', 'हो', 'जाव', 'त', 'एकर', 'जिम्मेदारी', 'हमरा', 'पर', 'मत', 'डालल', 'जाव़', 'कहानी', '२', 'जवार', 'भर', 'में', 'केहू', 'के', 'मजाल', 'ना', 'रहे', 'कि', 'भोला', 'पहलवान', 'का', 'सोझा', 'खड़ा', 'होखे', '.', 'जाने', 'केतना', 'मकानन', 'के', 'रूख', 'मरा', 'गइल', 'रहे', '।'], 'labels': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]}


In [ ]:
def has_entity(example):
    return any(label != 6 for label in example["labels"])

filtered_dataset = new_dataset.filter(has_entity)

Filter:   0%|          | 0/10370 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4486 [00:00<?, ? examples/s]

In [ ]:
print(len(filtered_dataset["train"]))

3051


In [ ]:
print(len(new_dataset["train"]))

10370


In [ ]:
tokenized_filtered_datasets = filtered_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3051 [00:00<?, ? examples/s]

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification

args = TrainingArguments(
    output_dir="./bhojpuri-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_filtered_datasets["train"],
    eval_dataset=tokenized_filtered_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-32-b6c3079b1639>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rijulsingla57 (rijulsingla57-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,0.149149,"{'precision': 0.7275747508305648, 'recall': 0.6347826086956522, 'f1': 0.6780185758513931, 'number': 690}","{'precision': 0.3076923076923077, 'recall': 0.2222222222222222, 'f1': 0.2580645161290323, 'number': 36}","{'precision': 0.7968281183026147, 'recall': 0.817322488459002, 'f1': 0.8069451980466632, 'number': 4549}",0.786551,0.789384,0.787965,0.952963
2,No log,0.125553,"{'precision': 0.7629382303839732, 'recall': 0.6623188405797101, 'f1': 0.7090768037238168, 'number': 690}","{'precision': 0.4, 'recall': 0.2777777777777778, 'f1': 0.3278688524590164, 'number': 36}","{'precision': 0.8562358276643991, 'recall': 0.8300725434161355, 'f1': 0.8429512222346244, 'number': 4549}",0.842868,0.804360,0.823164,0.961112
3,0.142300,0.124962,"{'precision': 0.7725040916530278, 'recall': 0.6840579710144927, 'f1': 0.7255956956187549, 'number': 690}","{'precision': 0.4782608695652174, 'recall': 0.3055555555555556, 'f1': 0.3728813559322034, 'number': 36}","{'precision': 0.8599427438890113, 'recall': 0.8584304242690701, 'f1': 0.8591859185918591, 'number': 4549}",0.847923,0.831848,0.839809,0.964321
4,0.142300,0.129587,"{'precision': 0.7363770250368189, 'recall': 0.7246376811594203, 'f1': 0.7304601899196492, 'number': 690}","{'precision': 0.4782608695652174, 'recall': 0.3055555555555556, 'f1': 0.3728813559322034, 'number': 36}","{'precision': 0.8625571770856023, 'recall': 0.8705209936249725, 'f1': 0.8665207877461706, 'number': 4549}",0.844701,0.847583,0.846139,0.965405
5,0.142300,0.131467,"{'precision': 0.744807121661721, 'recall': 0.7275362318840579, 'f1': 0.7360703812316717, 'number': 690}","{'precision': 0.4583333333333333, 'recall': 0.3055555555555556, 'f1': 0.3666666666666667, 'number': 36}","{'precision': 0.8639040348964013, 'recall': 0.8707408221587162, 'f1': 0.8673089555506898, 'number': 4549}",0.846867,0.848152,0.847509,0.965655


Trainer is attempting to log a value of "{'precision': 0.7275747508305648, 'recall': 0.6347826086956522, 'f1': 0.6780185758513931, 'number': 690}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.3076923076923077, 'recall': 0.2222222222222222, 'f1': 0.2580645161290323, 'number': 36}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7968281183026147, 'recall': 0.817322488459002, 'f1': 0.8069451980466632, 'number': 4549}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7629382303839732, 'recall': 0.6623188405797101, 'f

TrainOutput(global_step=955, training_loss=0.10216988768253027, metrics={'train_runtime': 474.4514, 'train_samples_per_second': 32.153, 'train_steps_per_second': 2.013, 'total_flos': 776415125854632.0, 'train_loss': 0.10216988768253027, 'epoch': 5.0})

In [ ]:
from collections import Counter

all_labels = [label for example in filtered_dataset["train"]["labels"] for label in example]
print(Counter(all_labels))

Counter({6: 52776, 2: 3689, 5: 1449, 0: 620, 3: 70, 4: 45, 1: 40})


In [ ]:
hindi_ner = pipeline(
    "ner",
    model="Davlan/xlm-roberta-base-wikiann-ner",
    aggregation_strategy="simple",
    device=-1
)

print("✓ Model loaded!\n")